## Accelerate Inference: Neural Network Pruning

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, regularizers
from tensorflow.keras.layers import *
from tqdm import tqdm
import time

print(tf.version.VERSION)

2.9.2


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# untar
!tar -xvzf /content/drive/MyDrive/10605MP/dataset.tar.gz
# load train
train_images = pickle.load(open('train_images.pkl', 'rb'))
train_labels = pickle.load(open('train_labels.pkl', 'rb'))
# load val
val_images = pickle.load(open('val_images.pkl', 'rb'))
val_labels = pickle.load(open('val_labels.pkl', 'rb'))

train_images.pkl
train_labels.pkl
val_images.pkl
val_labels.pkl


In [ ]:
# Define the neural network architecture (don't change this)

model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5), input_shape=(25,25,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 25, 25, 32)        896       
                                                                 
 activation (Activation)     (None, 25, 25, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 23, 23, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 23, 23, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 11, 11, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 11, 11, 32)        0         
                                                        

### Pre-train full model

In [ ]:
# you can use the default hyper-parameters for training, 
# and val accuracy ~59% after 25 epochs and > 63% after 50 epochs

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, batch_size=32, epochs=50, 
                    validation_data=(val_images, val_labels)) # train for 50 epochs, with batch size 32

Epoch 1/50


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


703/703 [==============================] - 7s 6ms/step - loss: 1.5464 - accuracy: 0.2883 - val_loss: 1.3906 - val_accuracy: 0.4028
Epoch 2/50
703/703 [==============================] - 4s 5ms/step - loss: 1.3741 - accuracy: 0.4085 - val_loss: 1.3081 - val_accuracy: 0.4416
Epoch 3/50
703/703 [==============================] - 4s 5ms/step - loss: 1.3005 - accuracy: 0.4469 - val_loss: 1.2282 - val_accuracy: 0.4800
Epoch 4/50
703/703 [==============================] - 4s 5ms/step - loss: 1.2544 - accuracy: 0.4768 - val_loss: 1.1862 - val_accuracy: 0.5030
Epoch 5/50
703/703 [==============================] - 4s 5ms/step - loss: 1.2159 - accuracy: 0.4975 - val_loss: 1.1651 - val_accuracy: 0.5160
Epoch 6/50
703/703 [==============================] - 4s 5ms/step - loss: 1.1844 - accuracy: 0.5155 - val_loss: 1.1416 - val_accuracy: 0.5224
Epoch 7/50
703/703 [==============================] - 5s 7ms/step - loss: 1.1515 - accuracy: 0.5309 - val_loss: 1.0883 - val_accuracy: 0.5477
Epoch 8/50
703/70

In [ ]:
results = model.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 8ms/step - loss: 0.7483 - accuracy: 0.7085


In [ ]:
model.save_weights("/content/drive/MyDrive/10605MP/pre_trained_weights.h5")

### Pruning

In [ ]:
layers = ['conv2d', 'conv2d_1', 'conv2d_2', 'conv2d_3', 'dense', 'dense_1']
activations = ['activation', 'max_pooling2d', 'activation_2', 'max_pooling2d_1', 'activation_4']

tf.random.set_seed(10605)
IMAGE_SAMPLE_SIZE = 500
LOCATION_SAMPLE_SIZE = 10

image_sample = train_images[tf.random.uniform(shape=[IMAGE_SAMPLE_SIZE], maxval=train_images.shape[0], dtype=tf.int64)]

In [ ]:
def collect_training_ex(model, layer_idx):
    cur_layer = model.get_layer(activations[layer_idx])
    next_layer = model.get_layer(layers[layer_idx + 1])
    model_till_then = models.Model(model.input, (cur_layer.output, next_layer.output))
    weights = model.get_weights()

    x, y = model_till_then(image_sample)

    if layer_idx == 1:
        # only 'conv2d_2' has zero padding
        x = tf.pad(x, tf.constant([[0, 0], [1, 1], [1, 1], [0, 0]]), 'CONSTANT')
    y -= weights[layer_idx * 2 + 3] # bias

    C = x.shape[-1]

    x_hat = []
    y_hat = []

    if layer_idx < 3:
        # conv2d_1, conv2d_2
        for image_i in tqdm(range(x.shape[0])):
            for i in range(LOCATION_SAMPLE_SIZE):
                h_i = tf.random.uniform(shape=[], maxval=y.shape[1], dtype=tf.int64)
                w_i = tf.random.uniform(shape=[], maxval=y.shape[2], dtype=tf.int64)
                x_ = tf.reshape(x[image_i, h_i:h_i+3, w_i:w_i+3, :], [-1, C])
                channel_i = tf.random.uniform(shape=[], maxval=y.shape[-1], dtype=tf.int64)
                w_ = tf.reshape(weights[layer_idx * 2 + 2][:, :, :, channel_i], [-1, C])
                x_hat.append(tf.reduce_sum(x_ * w_, axis=0))
                y_hat.append(y[image_i, h_i, w_i, channel_i])
    elif layer_idx == 3:
        # conv2d_3
        for image_i in tqdm(range(x.shape[0])):
            x_ = tf.reshape(x[image_i, :, :, :], [-1, C])
            for i in range(LOCATION_SAMPLE_SIZE):
                channel_i = tf.random.uniform(shape=[], maxval=y.shape[-1], dtype=tf.int64)
                w_ = tf.reshape(weights[layer_idx * 2 + 2][:, channel_i], [-1, C])
                x_hat.append(tf.reduce_sum(x_ * w_, axis=0))
                y_hat.append(y[image_i, channel_i])
    else:
        # dense
        for image_i in tqdm(range(x.shape[0])):
            x_ = x[image_i, :]
            for i in range(LOCATION_SAMPLE_SIZE):
                channel_i = tf.random.uniform(shape=[], maxval=y.shape[-1], dtype=tf.int64)
                w_ = weights[layer_idx * 2 + 2][:, channel_i]
                x_hat.append(x_ * w_)
                y_hat.append(y[image_i, channel_i])

    x_hat = np.array(x_hat)
    y_hat = np.array(y_hat).reshape(-1, 1)

    m = IMAGE_SAMPLE_SIZE * LOCATION_SAMPLE_SIZE
    assert x_hat.shape == (m, C)
    assert y_hat.shape == (m, 1)

    return x_hat, y_hat

In [ ]:
def select_channel(x_hat, compression_rate):
    C = x_hat.shape[-1]
    pruned = set()
    I = set(range(C))
    x_sum = 0
    num_pruned_channels = C * (1 - compression_rate)
    while len(pruned) < num_pruned_channels:
        min_value = float('inf')
        for i in I:
            value = tf.reduce_sum(tf.square(x_sum + x_hat[:, i]))
            if value < min_value:
                min_value = value
                min_i = i
        I.remove(min_i)
        pruned.add(min_i)
        x_sum += x_hat[:, min_i]
    selected = sorted(set(range(C)) - pruned)
    pruned = sorted(pruned)
    return pruned, selected

In [ ]:
def compress_model(model, compression_rate):
    start = time.time()
    
    # get the weights
    weights = model.get_weights()

    for layer_idx in range(1, len(layers) - 1):

        # prune
        model.trainable = False

        # collect training examples
        x_hat, y_hat = collect_training_ex(model, layer_idx)

        # greedy channel selection
        pruned, selected = select_channel(x_hat, compression_rate)

        # prune the corresponding filters in current layer
        if layer_idx < 4:
            weights[layer_idx * 2][:, :, :, pruned] = 0
        else:
            weights[layer_idx * 2][:, pruned] = 0

        # prune the corresponding dimensions in next layer
        if layer_idx < 3:
            weights[layer_idx * 2 + 2][:, :, pruned, :] = 0
        else:
            weights[layer_idx * 2 + 2][pruned, :] = 0

        # minimize reconstruction error by weighing selected channels
        x_selected = x_hat[:, selected]
        try:
            w = tf.matmul(tf.linalg.inv(tf.matmul(tf.transpose(x_selected), x_selected)), tf.matmul(tf.transpose(x_selected), y_hat))
            if layer_idx < 3:
                weights[layer_idx * 2 + 2][:, :, selected, :] *= w
            else:
                weights[layer_idx * 2 + 2][selected, :] *= w
        except tf.errors.InvalidArgumentError:
            # not invertible
            pass

        model.set_weights(weights)

        # fine-tune
        model.trainable = True
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
        model.fit(train_images, train_labels, batch_size=32, epochs=2,
                        validation_data=(val_images, val_labels))

    end = time.time()
    print(f"Time: {end - start}")

    model.save_weights(f"/content/drive/MyDrive/10605MP/my_model_weights_{compression_rate}.h5")

In [ ]:
def evaluate_model(model):
    # evaluate again to see how the accuracy changes
    accuracy = model.evaluate(val_images, val_labels, batch_size=128)[1]

    weights = model.get_weights()
    sparsity = 1 - sum(map(tf.math.count_nonzero, weights)) / sum(map(lambda x: x.size, weights))

    score = (accuracy + sparsity) / 2
    
    return accuracy, sparsity, score

In [ ]:
for compression_rate in np.arange(0., 1., 0.05):
    model.load_weights("/content/drive/MyDrive/10605MP/pre_trained_weights.h5")
    compress_model(model, compression_rate)

100%|██████████| 500/500 [00:25<00:00, 19.40it/s]


Epoch 1/2


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


703/703 [==============================] - 6s 6ms/step - loss: 1.5128 - accuracy: 0.3204 - val_loss: 1.3657 - val_accuracy: 0.4083
Epoch 2/2
703/703 [==============================] - 4s 5ms/step - loss: 1.3458 - accuracy: 0.4226 - val_loss: 1.2714 - val_accuracy: 0.4598


100%|██████████| 500/500 [00:18<00:00, 26.86it/s]


...

Epoch 1/2
703/703 [==============================] - 4s 5ms/step - loss: 0.6523 - accuracy: 0.7539 - val_loss: 0.7546 - val_accuracy: 0.7105
Epoch 2/2
703/703 [==============================] - 4s 5ms/step - loss: 0.6457 - accuracy: 0.7556 - val_loss: 0.7436 - val_accuracy: 0.7101


100%|██████████| 500/500 [00:08<00:00, 59.15it/s]


Epoch 1/2
703/703 [==============================] - 5s 6ms/step - loss: 0.6667 - accuracy: 0.7485 - val_loss: 0.7562 - val_accuracy: 0.7042
Epoch 2/2
703/703 [==============================] - 4s 5ms/step - loss: 0.6571 - accuracy: 0.7531 - val_loss: 0.7479 - val_accuracy: 0.7057


100%|██████████| 500/500 [00:07<00:00, 63.54it/s]


Epoch 1/2
703/703 [==============================] - 4s 5ms/step - loss: 0.6688 - accuracy: 0.7455 - val_loss: 0.7569 - val_accuracy: 0.7077
Epoch 2/2
703/703 [==============================] - 4s 5ms/step - loss: 0.6585 - accuracy: 0.7516 - val_loss: 0.7460 - val_accuracy: 0.7077
Time: 92.33249425888062


In [ ]:
for compression_rate in np.arange(0., 1., 0.05):
    model.load_weights(f"/content/drive/MyDrive/10605MP/my_model_weights_{compression_rate}.h5")
    print(','.join(map(lambda x: str(float(x)), [compression_rate, *evaluate_model(model)])))

20/20 [==============================] - 0s 7ms/step - loss: 1.4194 - accuracy: 0.3830
0.0,0.38297030329704285,0.514621382179774,0.44879584273840845
20/20 [==============================] - 0s 7ms/step - loss: 1.3694 - accuracy: 0.4123
0.05,0.4122772216796875,0.49942236306631604,0.45584979237300177
20/20 [==============================] - 0s 9ms/step - loss: 1.2650 - accuracy: 0.4602
0.1,0.4601980149745941,0.45720848729957686,0.4587032511370855
20/20 [==============================] - 0s 6ms/step - loss: 1.2104 - accuracy: 0.4947
0.15000000000000002,0.49465346336364746,0.41850934253954497,0.4565814029515962
20/20 [==============================] - 0s 6ms/step - loss: 1.1331 - accuracy: 0.5319
0.2,0.5318812131881714,0.39875331614195875,0.46531726466506507
20/20 [==============================] - 0s 5ms/step - loss: 1.0818 - accuracy: 0.5640
0.25,0.5639603734016418,0.353151199208005,0.4585557863048234
20/20 [==============================] - 0s 5ms/step - loss: 1.0495 - accuracy: 0.5782


In [3]:
# running this cell will immediately download a file called 'my_model_weights.h5'
from google.colab import files
files.download("/content/drive/MyDrive/10605MP/my_model_weights_0.5.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>